# Trabalho Data Minig e Machine Learning II

O dataset Costa Rican Household Poverty Level Prediction tem como objetivo classificar as casas por meio de quatro níveis de renda: 1 - que indica extrema pobreza, 2 - pobreza moderada, 3 - lares vulneráveis e 4 - casas não vulneráveis. O objetivo é classificar essas casas dentro destas quatro categorias com o objetivo de identificar as famíliar que mais necessitam de auxílio do governo da Costa Rica.

Dessa forma, primeiramente são transformadas as variáveis do tipo object para int - menos as variáveis do tipo código.

Após isso, é feito o tratamento dos campos nulos das variáveis, imputando os valores de acordo com outras variáveis. 

Então é feito uma análise exploratória dos dados, verificando a relação dos níveis de renda com as outras variáveis. Aí também é descoberta a correlação alta entre variáveis explicativas, e isso pode acabar enviesando os modelos, portanto decidiu-se por retirar estas variáveis do dataset.

Após a análise exploratória, testou-se os modelos de classificação: Random Forest, XGBoost e AdaBoost.

In [ ]:
# Bibliotecas: 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import scikitplot as skplt
from sklearn.metrics import accuracy_score
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Carregando os dados

df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

In [ ]:
# Juntando os dataframes

df_all = df.append(test)

df_all.shape

In [ ]:
# Verificando as cinco primeiras linhas do dataset: 

df_all.head()

# Transformando variáveis:

In [ ]:
# Quais colunas do dataframe são do tipo object

df_all.select_dtypes('object').head()

In [ ]:
# Olhando a coluna dependency

df_all['dependency'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefa

df_all['edjefa'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefe

df_all['edjefe'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0 nas colunas edjefa e edjefe

mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

In [ ]:
# Olhando a coluna dependency

df_all['dependency'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0 na coluna dependency

df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
# Verificando se ainda há colunas do dataframe do tipo object para transformar:

df_all.select_dtypes('object').head()

Não iremos transformar as variáveis do tipo id.

In [ ]:
# Verificando algumas informações do dataset:

df_all.info()

# Tratamento dos Valores Nulos

In [ ]:
# Verificando a porcentagem total de valores nulos no dataset:

total = np.product(df_all.shape)
miss_values = df_all.isnull().sum().sum()
porcentagem_total = (miss_values/total) * 100
print(f'A porcentagem total de valores nulos do dataset é de: {(porcentagem_total)}')

In [ ]:
# Verificando a porcentagem de valores nulos por variável:

total_num = df_all.isnull().sum().sort_values(ascending = False)

perc = df_all.isnull().sum()/df_all.isnull().count()*100

perc1 = (round(perc,2).sort_values(ascending = False))

#Criando o dataframe: 

df_miss = pd.concat([total_num, perc1], axis = 1, keys = ['Total de Valores Nulos', 'Porcentagem %']).sort_values(by = 'Porcentagem %', ascending = False)

top_miss = df_miss[df_miss['Porcentagem %'] > 0]
top_miss.reset_index(inplace = True)

top_miss

**Colunas com valores nulos:** 

rez_esc: Years behind in school

v18q1: number of tablets household owns

v2a1: Monthly rent payment

meaneduc: average years of education for adults (18+)

SQBmeaned: square of the mean years of education of adults (>=18) in the household


Destas variáveis, é possível perceber que as que mais têm valores nulos são as primeiras, então vamos investigar cada variável. 








In [ ]:
# Variável rez_esc:

df_all.rez_esc.value_counts()

A variável rez_esc tem 82% de seus campos nulos, portanto veremos como tratá-la. É possível que as famílias que estão com valores nulos nestes campos, simplesmente não tenham crianças na escola. Portanto, é interessante verificar a idade daqueles que têm campos nulos nesta variável e dos que não têm campos nulos. Primeiro, vamos verificar a idade dos que não têm campos nulos: 

In [ ]:
df_all.loc[df_all['rez_esc'].notnull()]['age'].describe()

Por meio destas estatísticas descritivas é possivel perceber que a média da idade de crianças que estão na escola é de 12 anos. Além disso, a idade mínima é de 7 anos e a idade máxima de 17 anos. Pode-se assumir que pessoas que tenham menos de 7 anos e mais de 17 anos não estejam na escola. Agora, vamos verificar a idade daqueles que têm campos nulos nesta variável: 

In [ ]:
df_all.loc[df_all['rez_esc'].isnull()]['age'].describe()

Pode-se ver que a idade média é de 39 anos, a mediana de 38 e idade máxima de 97 anos. Assim, nesta variável, para aqueles que têm menos de 7 anos e mais de 19, iremos imputar 0 nos campos nulos. 

In [ ]:
# Tratamento da coluna rez_esc:

df_all.loc[((df_all['age'] > 19) | (df_all['age'] < 7)) & (df_all['rez_esc'].isnull()), 'rez_esc'] = 0

In [ ]:
df_all['rez_esc-missing'] = df_all['rez_esc'].isnull()

In [ ]:
# Tratando os outliers da coluna:

df_all.loc[df_all['rez_esc'] > 5, 'rez_esc'] = 5

In [ ]:
df_all.loc[df_all['rez_esc'].isnull()]['age'].describe()

In [ ]:
df_all['rez_esc'].fillna(0, inplace = True)

In [ ]:
# Variável v18q1:

df_all.v18q1.value_counts()

In [ ]:
# Vamos verificar outra variável relacionada
# v18q: owns a tablet (0 indica que a pessoa não possui tablet/ 1 indica que a pessoa possui tablet)

df_all.v18q.value_counts()

25.468 pessoas não têm tablets, sendo que essa é a mesma quantidade de campos nulos na variável v18q1 (quantidade de tablets que possui). Dessa forma, pode-se perceber que, possivelmente, este campo não foi preenchido por aquelas pessoas que não têm tablet. Portanto, vamos imputar o número 0 nos campos nulos da variável v18q1. 

In [ ]:
# Imputando o valor 0 na variável v18q1:

df_all['v18q1'].fillna(0, inplace = True)

In [ ]:
sns.distplot(a = df_all['v18q1'], kde = False)

In [ ]:
# Variável v2a1: 

df_all.v2a1.value_counts()

Pelo fato de a variável possuir muitos valores nulos, mas ser importante para definir o nível de renda da pessoa, decidiu-se por imputar -1 nos campos nulos da variável.

In [ ]:
# Verificando quantas pessoas alugam as casas (tipovivi3, = 1 rented):

df_all.tipovivi3.value_counts().sort_values()

Apenas 5.652 pessoas alugam casas. Talvez seja por isso que os 24.263 valores da variável de pagamento mensal de aluguel estejam nulos.

In [ ]:
# Verificando quantas pessoas possuem casa própria (tipovivi1, =1 own and fully paid house):

df_all.tipovivi1.value_counts().sort_values()

Indica que 20.844 pessoas possuem casa própria, totalmente paga.

In [ ]:
# Quantidade de casas que ainda estão sendo pagas (tipovivi2, "=1 own,  paying in installments"):

df_all.tipovivi2.value_counts().sort_values()

Ao somar a quantidade de casas que já estão pagas e que estão sendo pagas, tem-se 24.342, então, possivelmente, os valores nulos de v2a1 sejam referentes às casas próprias. 

In [ ]:
# tipovivi4, =1 precarious

df_all.tipovivi4.value_counts()

In [ ]:
# tipovivi5, "=1 other(assigned,  borrowed)"

df_all.tipovivi5.value_counts()

Após verificar todas as quantidades de cada tipo de casa do dataset, verificou-se que provavelmente a quantidade de valores nulos de v2a1 seja correspondente ao número de casas própria pagas e que estão sendo pagas (tipovivi1 e tipovivi2). Assim, decidiu-se por imputar o número zero nos campos nulos da variável refente ao pagamento de aluguel (v2a1).

In [ ]:
# Imputando o valor 0 na variável v2a1:

df_all['v2a1'].fillna(0, inplace=True)

As variáveis SQBmeaned e meaneduc têm a mesma quantidade de valores nulos, portanto vamos imputar -1 nestas observações:

In [ ]:
# Tratamento SQmeaned e meaneduc:

df_all['SQBmeaned'].fillna(-1, inplace=True)
df_all['meaneduc'].fillna(-1, inplace=True)

Vamos verificar se ainda há valores nulos no dataset: 

In [ ]:
# Verificando a porcentagem total de valores nulos no dataset:

total = np.product(df_all.shape)
miss_values = df_all.isnull().sum().sum()
porcentagem_total = (miss_values/total) * 100
print(f'A porcentagem total de valores nulos do dataset é de: {(porcentagem_total)}')

Essa porcentagem se refere à Target.

# Análise Exploratória dos Dados

In [ ]:
# Verificando a quantidade de casas do dataset:

df_all.idhogar.nunique()

In [ ]:
# Verificando a quantidade de chefes de família: 

df_all.parentesco1.value_counts()

Temos o id de 10.340 casas e 10.307 chefes de família, ou seja, pode ser que em algumas casas não há um chefe ou chefa de família.

In [ ]:
# Verificando quantas pessoas têm em quantas casas:
# r4t3: Total persons in the household

(
    df_all.groupby('r4t3').idhogar.nunique()
    .reset_index()
    .rename(columns = {'r4t3':'Quantidade de Pessoas na Casa', 'idhogar':'Quantidade de Casas'})
    .sort_values(by = ['Quantidade de Casas'], ascending = True)
    .reset_index()
    .drop(['index'], axis = 1)
    
)

In [ ]:
# Verificando a quantidade de pessoas que vivem na casa:
# tamviv, number of persons living in the household

(
    df_all.groupby('tamviv').idhogar.nunique()
    .reset_index()
    .rename(columns={'tamviv':'Quantidade de Pessoas que vivem na Casa', 'idhogar':'Quantidade de Casas'})
    .sort_values(by=['Quantidade de Casas'], ascending=True)
    .reset_index()
    .drop(['index'], axis = 1)
    
)

Pode-se perceber que a quantidade de pessoas que estão na casa é diferente da quantidade de pessoas que vivem na casa. 

In [ ]:
# Verificando a quantidade de casas por nível de renda: 

(
df_all.groupby('Target').idhogar.count()
)

In [ ]:
# Visualizando a distribuição da Target por meio de um gráfico:

niveis_renda = df_all['Target'].value_counts().sort_index()

from collections import OrderedDict

cores = OrderedDict({1: 'red', 2: 'orange', 3: 'blue', 4: 'green'})

niveis_renda.plot.bar(figsize = (8, 6), 
                      color = cores.values(),
                      edgecolor = 'k', linewidth = 2)

plt.xlabel('Níveis de Renda')
plt.ylabel('Quantidade')
plt.xticks(rotation = 70) 
plt.title('Distribuição dos Níveis de Renda')

Pode-se perceber que há muito mais casas que são do tipo não vulneráveis do que os outros os outros tipos de lares. Dessa forma, teremos que balancear os dados antes de jogá-los nos modelos de machine learning.

In [ ]:
# Verificando a quantidade de meninos mais jovens que 12 anos de idade por casa:
# r4h1, Males younger than 12 years of age

(
    df_all.groupby('r4h1').idhogar.nunique()
    .reset_index()
    .rename(columns={'r4h1':'Meninos mais jovens que 12 anos de idade','idhogar':'Quantidade de Casas'})
    
)

In [ ]:
# Verificando a quantidade de meninos de 12 anos e mais velhos:
# r4h2, Males 12 years of age and older

(
    df_all.groupby('r4h2').idhogar.nunique()
    .reset_index()
    .rename(columns={'r4h2':'Homens com 12 anos de idade ou mais velhos','idhogar':'Quantidade de Casas'})
    
)

In [ ]:
# Verificando o total de homens na casa:
# r4h3, Total males in the household

(
    df_all.groupby('r4h3').idhogar.nunique()
    .reset_index()
    .rename(columns={'r4h3':'Total de Homens na Casa','idhogar':'Quantidade de Casas'})
    
)

In [ ]:
# Quantidade de meninas mais jovens que 12 anos de idade:
# r4m1, Females younger than 12 years of age

(
    df_all.groupby('r4m1').idhogar.nunique()
    .reset_index()
    .rename(columns={'r4m1':'Meninas mais jovens que 12 anos de idade','idhogar':'Quantidade de Casas'})
    
)

In [ ]:
# Meninas de 12 anos de idade e mais velhas: 
# r4m2, Females 12 years of age and older

(
    df_all.groupby('r4m2').idhogar.nunique()
    .reset_index()
    .rename(columns={'r4m2':'Meninas de 12 anos de idade ou mais velhas','idhogar':'Quantidade de Casas'})
    
)

In [ ]:
# Total de mulheres na casa:
# r4m3, Total females in the household

(
    df_all.groupby('r4m3').idhogar.nunique()
    .reset_index()
    .rename(columns={'r4m3':'Total de Mulheres na Casa','idhogar':'Quantidade de Casas'})
    
)

In [ ]:
# Visualizando a quantidade de pessoas mais jovens que 12 anos na casa:

plt.figure(figsize = (8, 6))
sns.distplot(a = df_all['r4t1'])
plt.title('Número de pessoas mais jovens que 12 anos na casa')
plt.xlabel('Quantidade')

In [ ]:
# Visualizando a quantidade de pessoas de 12 anos ou mais velhas na casa:

plt.figure(figsize = (8, 6))
sns.distplot(a = df_all['r4t2'])
plt.title('Número de pessoas de 12 anos ou mais velhas na casa')
plt.xlabel('Quantidade')

In [ ]:
# Visualizando o total de pessoas na casa: 
# r4t3, Total persons in the household

plt.figure(figsize = (8, 6))
sns.distplot(a = df_all['r4t3'])
plt.title('Total de Pessoas na Casa')
plt.xlabel('Quantidade')

In [ ]:
# Verificando a quantidade de anos de educação de chefas em relação ao nível de renda: 
# 1 = extreme poverty
# 2 = moderate poverty
# 3 = vulnerable households
# 4 = non vulnerable households

edjefas_renda = pd.pivot_table(
df_all, 
columns = 'Target',
index = 'edjefa',
values = 'Id',
aggfunc = {'Id':'count'}
)

edjefas_renda

* A maior concentração de chefas, em todos os níveis, está em zero anos de educação. 

* Nível de pobreza extrema: apenas uma mulher tem 16 anos de educação.

* Nível de casas não vulneráveis: apenas cinco chefas têm 21 anos de educação.  

In [ ]:
# Verificando a quantidade de anos de educação de chefes em relação ao nível de renda: 
# 1 = extreme poverty
# 2 = moderate poverty
# 3 = vulnerable households
# 4 = non vulnerable households

edjefes_renda = pd.pivot_table(
df_all, 
columns = 'Target',
index = 'edjefe',
values = 'Id',
aggfunc = {'Id':'count'}
)

edjefes_renda

* Assim como edjefas, a maior concentração de chefes está naqueles que não têm anos de educação. 

* Pode-se ver que no nível de não vulnerabilidade das casas, há uma maior distribuição que nos outros níveis, sendo que há 43 chefes com 21 anos de educação. Mesmo assim, por meio das tabelas não é possível verificar se edjefes ou edjefas possuem maior nível de educação por renda. 

In [ ]:
# Verificar a quantidade de quartos pela média de aluguel:

room_rent = (
    df_all.groupby('rooms').v2a1.mean()
).reset_index()

room_rent.rename(columns = {'rooms':'Quantidade de Quartos', 'v2a1':'Média do Aluguel Mensal'})

In [ ]:
# Verificando a correlação entre variáveis: 
# Correlação tamanho da casa: 

corr_casa = df_all[['tamhog','tamviv','hhsize','r4t3']]

corr_casa.corr()

Por meio desse quadro de correlação, é possível perceber que as variáveis são bastante correlacionadas.

In [ ]:
# Visualizando a correlação: 

sns.lmplot('tamhog', 'hhsize', data = df_all, fit_reg = True);
plt.title('Size of the Household vs Household Size')

Visualizando a correlação entre tamhog e hhsize pode-se ver que se trata de uma correlação perfeita.

In [ ]:
# Visualizando a correlação: 

sns.lmplot('tamviv', 'r4t3', data = df_all, fit_reg = True);
plt.title('Number of Persons living in the Household vs Total Persons in the Household')

Por meio do gráfico de dispersão é possível ver que as variáveis são bastante correlacionadas também, portanto decidiu-se por retirar as variáveis hhsize e r4t3.

In [ ]:
# Retirando as variáveis hhsize e r4t3: 

df_all = df_all.drop(['hhsize','r4t3'], axis = 1)

In [ ]:
# Verificando a correlação entre a variável idade e idade ao quadrado:

corr_idade = df_all[['age','SQBage']].corr()
print(f'A correlação entre age e SQBage é de: \n{(corr_idade)}')

In [ ]:
# Visualizando a correlação: 

sns.lmplot('age', 'SQBage', data = df_all, fit_reg = False);
plt.title('Squared Age versus Age')

Pode-se perceber que as duas variáveis são bastante correlacionadas, portanto decidiu-se por retirar a variável age do dataset. 

In [ ]:
# Retirando a variável age: 

df_all = df_all.drop(['age'], axis = 1)

# Modelo de Classificação Random Forest 

In [ ]:
# Separando as colunas para treinamento:

feats = [c for c in df_all.columns if c not in ['Id', 'idhogar','Target']]

In [ ]:
# Separar os dataframes:

train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

In [ ]:
# Instanciando o random forest classifier

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=700,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,
                            verbose=0, class_weight='balanced')

In [ ]:
# Treinando o modelo

rf.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado

test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
# Vamos verificar as previsões

test['Target'].value_counts(normalize=True)

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)

fig = plt.figure(figsize=(15, 20))

pd.Series(rf.feature_importances_, index = feats).sort_values().plot.barh()

plt.title('Importância de cada Variável no Modelo Random Forest')

No modelo de Random Forest, as duas variáveis que foram consideradas como as mais importante para a classificação são aquelas relacionadas à educação, meaneduc e SQBmeaned.

In [ ]:
# Matriz de Confusão

skplt.metrics.plot_confusion_matrix(train['Target'], rf.predict(train[feats])) 

* No primeiro nível de renda - extrema pobreza - o modelo predisse corretamente que 650 pessoas fazem parte do nível 1, mas errou ao predizer que 24 pessoas faziam parte do nível 2, 20 do nível 3 e 61 do nível 4. 

* No segundo nível de renda - pobreza moderada - o modelo acertou ao prever 1202 pessoas pertencentes ao segundo nível, mas errou ao predizer 87 como pertencentes ao nível 1, 117 ao nível 3 e 191 ao nível 4.

* No terceiro nível - casas vulneráveis - o modelo predisse corretamente 929 pessoas, errando ao predizer 59 como pertencentes ao nível 1, 44 ao nível 2 e 177 ao nível 4.

* No último nível - casas não vulneráveis - o modelo predisse corretamente 4664 pessoas, mas errou ao predizer que 249 pessoas pertenciam ao primeiro nível, 414 ao segundo e 669 ao terceiro.

**Acurácia**

A métrica que diz quanto que o modelo acertou no total é a acurácia, para calculá-la é necessário somar os acertos do modelo e dividir pelo tamanho do dataset:

In [ ]:
# Acurácia: 

(650 + 1202 + 929 + 4664) / 9557

A acurácia do modelo indica o quanto ele está acertando do dataset, ou seja, neste caso, 77%.

In [ ]:
# Criando o arquivo para submissão

#test[['Id','Target']].to_csv('submission.csv', index=False)

Score do Kaggle: 0.43248

# Modelo XGBoost

In [ ]:
# Trabalhando com XGBoost

from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(n_estimators = 200, learning_rate = 0.09, random_state = 42, class_weight = 'balanced')

In [ ]:
xgb.fit(train[feats], train['Target'])

In [ ]:
test['Target'] = xgb.predict(test[feats]).astype(int)

In [ ]:
accuracy_score(train['Target'], xgb.predict(train[feats]))

A acurácia do modelo de XGBoost também foi de 77%.

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)

fig = plt.figure(figsize=(15, 20))

pd.Series(xgb.feature_importances_, index = feats).sort_values().plot.barh()

plt.title('Importância de cada Variável no Modelo XGBoost')

O modelo de XGBoost entendeu como variáveis mais importantes aquelas relacionadas à casa, como se a casa possui teto ou não, o tipo de piso da casa, se o piso é bom ou não, quantidade de tablets que possui na casa. 

In [ ]:
# Matriz de Confusão

skplt.metrics.plot_confusion_matrix(train['Target'], xgb.predict(train[feats])) 

* No primeiro nível de renda - extrema pobreza - o modelo predisse corretamente que 382 pessoas fazem parte do nível 1, mas errou ao predizer que 142 pessoas faziam parte do nível 2, 2 do nível 3 e 229 do nível 4. 

* No segundo nível de renda - pobreza moderada - o modelo acertou ao prever 895 pessoas pertencentes ao segundo nível, mas errou ao predizer 26 como pertencentes ao nível 1, 14 ao nível 3 e 662 ao nível 4.

* No terceiro nível - casas vulneráveis - o modelo predisse corretamente 319 pessoas, errando ao predizer 15 como pertencentes ao nível 1, 202 ao nível 2 e 673 ao nível 4, ou seja, errou mais que acertou neste nível.

* No último nível - casas não vulneráveis - o modelo predisse corretamente 5780 pessoas, mas errou ao predizer que 13 pessoas pertenciam ao primeiro nível, 199 ao segundo e 4 ao terceiro.

In [ ]:
# Criando o arquivo para submissão

#test[['Id','Target']].to_csv('submission.csv', index=False)

**Score Kaggle: 0.36531**  

# Modelo AdaBoost

In [ ]:
# Trabalhando com AdaBoost

from sklearn.ensemble import AdaBoostClassifier

In [ ]:
abc = AdaBoostClassifier(n_estimators = 200, learning_rate = 1.0, random_state = 42)

In [ ]:
abc.fit(train[feats], train['Target'])

In [ ]:
accuracy_score(train['Target'], abc.predict(train[feats]))

A acurácia do modelo de AdaBoost foi menor que nos outros modelos, sendo de 67%.

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)

fig = plt.figure(figsize=(15, 20))

pd.Series(abc.feature_importances_, index = feats).sort_values().plot.barh()

plt.title('Importância de cada Variável no Modelo AdaBoost')

O modelo de AdaBoost considera como variáveis mais importantes para explicar o modelo as variáveis relacionadas ao pagamento mensal de aluguel e aquelas relacionadas à educação. 

In [ ]:
# Matriz de Confusão

skplt.metrics.plot_confusion_matrix(train['Target'], abc.predict(train[feats])) 

* No primeiro nível de renda - extrema pobreza - o modelo predisse corretamente que 282 pessoas fazem parte do nível 1, mas errou ao predizer que 209 pessoas faziam parte do nível 2, 37 do nível 3 e 227 do nível 4.

* No segundo nível de renda - pobreza moderada - o modelo acertou ao prever 594 pessoas pertencentes ao segundo nível, mas errou ao predizer 190 como pertencentes ao nível 1, 121 ao nível 3 e 692 ao nível 4.

* No terceiro nível - casas vulneráveis - o modelo predisse corretamente 228 pessoas, errando ao predizer 85 como pertencentes ao nível 1, 265 ao nível 2 e 631 ao nível 4, ou seja, errou mais que acertou neste nível.

* No último nível - casas não vulneráveis - o modelo predisse corretamente 5362 pessoas, mas errou ao predizer que 96 pessoas pertenciam ao primeiro nível, 389 ao segundo e 149 ao terceiro.



In [ ]:
# Criando o arquivo para submissão

test[['Id','Target']].to_csv('submission.csv', index=False)

**Score do Kaggle: **

# Conclusões




* Verificando a acuária dos três modelos utilizados, pôde-se perceber que aquele que teve a maior acurácia foi o modelo de Random Forest. 

* O modelo que teve a maior classificação de score pelo Kaggle foi: 

* É importante mencionar que quando o modelo de Random Forest foi aplicado sem balancear os dados, o resultado do score obtido foi de apenas 0,36. Esse score só aumentou após balancear os dados. 

* Os maiores desafios enfrentados neste trabalho foram relacionados ao tratamento dos campos nulos das variáveis, e em relação às formas de tentar melhorar os modelos. 